### Can we classify each phase as the TSST (stress) vs non-stress phase for each subject? ###

In [97]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
cvx_path = os.path.abspath(os.path.join('..', '..', 'cvxEDA', 'src'))
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)
import pandas as pd
import random
import scipy.signal as ss
import sys

import tools.data_reader_wesad as dr
import tools.display_tools as dt
import tools.preprocessing as preprocessing
import train

from scipy.fft import fft, fftfreq, fftshift
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score, RepeatedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import normalize

import cvxopt.solvers
cvxopt.solvers.options['show_progress'] = False

import warnings
warnings.filterwarnings(
    "ignore", 
    category=RuntimeWarning
)


phases = dr.Phases.PHASE_ORDER


In [105]:
importlib.reload(train)
importlib.reload(dt)
importlib.reload(preprocessing)


metrics = [
    "bpm", 
    "rmssd", 
    "hf_rr", 
    "lf_rr", 
    "ibi", 
    # "mean_SCL", 
    # "SCR_rate"
]

phases = dr.Phases.PHASE_ORDER

x, y = train.Train_WESAD.get_wesad_data(metrics, phases, verbose=False, label_type="all")

models = {
    "SVM": SVC(C=1, gamma=0.01), 
    # "KNN": KNeighborsClassifier(n_neighbors=7)
}

results = {
    "SVM": [], 
    # "KNN": []
}
num_iters = 10
for _ in range(num_iters):
    out = train.train_predict(models, x, y, by_subject=False, show_classification=False)
    for model_name in results:
        results[model_name].append(out[model_name])

for model_name in results.keys():
    print(f"{model_name} accuracy over {num_iters} rounds: {np.mean(results[model_name])}")

SVM accuracy over 10 rounds: 0.8
